In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

## Annotation file

The lncRNA transcripts were given as a .fasta file.

Firstly, sequences in fasta files were renamed, because the names were too long. The renamed fasta was aligned to the genome, turned to bed and gtf produced out of the bed file was used. 

In [ ]:
with open("merged_lncRNAs_transcripts_filtered_nonredundant.out_30_1_plus", "r") as infa:
    with open("merged_lncRNAs_transcripts_filtered_nonredundant_new.fasta", "w") as outfa:
        with open("merged_lncRNAs_transcripts_filtered_nonredundant_tocutfasta.csv", "w") as pot:
            i = 0
            for line in infa:
                if line.startswith(">"):
                    i += 1
                    outfa.write(">lnc_tr")
                    outfa.write(str(i) + "\n")
                    pot.write(line.strip() + ";" + ">lnc_tr" + str(i) + "\n") # table of new and old names matching 
                else:
                    outfa.write(line)

Flair needs reads, gtf annotation and a reference. Gtf annotation was produced from fasta file of lncRNAs with **fqtobed.sh** script. Then bed12ToGTF from https://github.com/gold-lab/bed12ToGTF was used to produce gtf. The resulted file lacked gene_id, so the following commands were used.

In [ ]:
!sed 's/transcript/gene/' lnctogtf.gtf > lnctogtf_genes.gtf
!cut -f 9 lnctogtf_genes.gtf | sed 's/;/";/g' | sed 's/number /number "/' > lnctogtf_genes_column
!sed 's/;/; transcript_id/' lnctogtf_genes_column > lnctogtf_genes_column2
!awk '{print $1, $2, $3, $2, $4, $5, $6}' lnctogtf_genes_column2 > lnctogtf_genes_column_fin

In [ ]:
gtf = pd.read_csv('lnctogtf_genes.gtf', sep='\t', header = None)
fin = pd.read_csv('lnctogtf_genes_column_fin', sep='\t', header = None)
gtf.iloc[:,8] = fin.iloc[:,0]
gtf.to_csv("lnc_annot_1.gtf", sep = "\t", header=False, index=False)
!sed 's/""/"/g' lnc_annot_1.gtf | sed 's/"gene/gene/' | sed 's/;   "/;/' | sed 's/; "/;/' > lnc_annot.gtf

## Reads

Basecalling was performed using Guppy.

#### guppy_basecaller --input_path fast5/g3 --num_callers 22 --save_path /media/fastq/moss/g3 --flowcell FLO-MIN106 --kit SQK-RNA002 --qscore_filtering -r


.bed12 files were produced with **fqtobed.sh** script.
Then all .bed files were merged into 1 file in the following manner:

#### cat p?.bed >> guppy4all.bed

Then the file was sorted:

#### bedtools sort -i guppy4all.bed > guppy4all_sort.bed

All .fastq reads were copied to a single file for each replication and into a file with all reads of all replications.

#### cat guppy4/g2/*.fastq >> g2.fastq
#### cat guppy4/g2/*.fastq >> all_seqs.fastq

## Flair

Flair of the latest version from https://github.com/BrooksLabUCSC/flair was used. However, an older version was used for **flair correct** command. The pipeline:

In [ ]:
!sudo docker exec 537efdb1ac27 python3 /usr/local/flair/flair.py correct -t 18 -o lnc -q guppy4all_sort.bed -g moss_ref.fasta -f lnc_annot.gtf -c sizes.genome
!python flair.py collapse -f lnc_annot.gtf --generate_map -o lnc -q lnc_corrected.bed -g moss_ref.fasta -r all_seqs.fastq -s 1 -t 18
!python ~/anna/flair/flair.py quantify -t 18 -o lnc_newgtf2 -r reads_manifest.tsv -i lnc.isoforms.fa
!grep lnc lnc_newgtf2 | sed 's/.*_lnc/lnc/' > lnc_newgtf2_codes.tsv

In [8]:
flair = pd.read_csv('lnc_newgtf2_codes.tsv', sep='\t', header = None)
flair.head()

,0,1,2,3,4,5,6,7
0,lnc_tr14435,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,lnc_tr4765,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,lnc_tr15475,0.0,0.0,7.0,0.0,0.0,0.0,0.0
3,lnc_tr8473,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,lnc_tr12486,0.0,0.0,0.0,0.0,11.0,1.0,13.0


In [9]:
# Now we can return full transcript names
names = pd.read_csv('lnc_tr_names_tomynames.csv', sep=';', header=None)
names.head()

,0,1
0,>CNT2063571,lnc_tr1
1,>CNT2063572,lnc_tr2
2,>CNT2063573,lnc_tr3
3,>CNT2063574,lnc_tr4
4,>CNT2063575,lnc_tr5


In [10]:
flair = flair.rename(columns = {0: "ids",1: "g1_gam",2: "g2_gam",3: "g3_gam",4: "g4_gam", 5: "p1_prot", 6: "p2_prot",7: "p3_prot"})
names = names.rename(columns={1: "ids", 0: "names"})
flair = flair.merge(names, how="inner", on="ids")

In [33]:
flair["source"] = "flair"
flair.head()

,ids,g1_gam,g2_gam,g3_gam,g4_gam,p1_prot,p2_prot,p3_prot,names,g1_rpm,g2_rpm,g3_rpm,g4_rpm,p1_rpm,p2_rpm,p3_rpm,source
0,lnc_tr14435,0.0,0.0,1.0,0.0,0.0,0.0,0.0,>XR_002972246.1 rna-XR_002972246.1_NCBI_ncRNA,0.0,0.00000,1.298701,0.0,0.000000,0.0,0.000000,flair
1,lnc_tr14435,0.0,3.0,0.0,0.0,1.0,0.0,0.0,>XR_002972246.1 rna-XR_002972246.1_NCBI_ncRNA,0.0,3.05499,0.000000,0.0,0.626174,0.0,0.000000,flair
2,lnc_tr4765,0.0,0.0,0.0,0.0,1.0,0.0,1.0,>lcl|Ppatens_Pp3c1_40640V3.1,0.0,0.00000,0.000000,0.0,0.626174,0.0,0.552181,flair
3,lnc_tr15475,0.0,0.0,7.0,0.0,0.0,0.0,0.0,>XR_002970609.1 rna-XR_002970609.1_NCBI_ncRNA,0.0,0.00000,9.090909,0.0,0.000000,0.0,0.000000,flair
4,lnc_tr8473,0.0,0.0,0.0,0.0,1.0,0.0,1.0,>lcl|Ppatens_Pp3c6_15880V3.1,0.0,0.00000,0.000000,0.0,0.626174,0.0,0.552181,flair


In [16]:
# The number of reads is from samtools flagstat
flair["g1_rpm"] = flair.g1_gam/0.393
flair["g2_rpm"] = flair.g2_gam/0.982
flair["g3_rpm"] = flair.g3_gam/0.77
flair["g4_rpm"] = flair.g4_gam/0.88
flair["p1_rpm"] = flair.p1_prot/1.597
flair["p2_rpm"] = flair.p2_prot/0.5656
flair["p3_rpm"] = flair.p3_prot/1.811

## featureCounts

Flair cannot correctly identify isoforms with no introns. For monoexon features identification we used **featureCounts**. Monoexon features were extracted like this:

In [ ]:
with open("lnc_annot.gtf", "r") as gtf:
    with open("lnc_annot_1ex.gtf", "w") as mono:
        for line in gtf.readlines():
            line = line.split("\t")
            if line[2] == "gene":
                gene = line
            else:
                if line[3] == gene[3] and line[4] == gene[4]: # first exon is the same as the whole transcript
                    mono.write("\t".join(gene))

featureCounts takes annotation in a .saf format. We used a threshold of 75% for the feature to be covered by read.

In [ ]:
!awk '{print $10, $3, "\t" $1, "\t", $4, "\t", $5, "\t", $7}' lnc_annot.gtf > lnc_annotm.saf
!sed 's/ //g' lnc_annotm.saf > lnc_annot.saf
!./featureCounts -a lnc_annot_1e.saf -F SAF -s 1 -L -O -o lnc_monoexon75 --fracOverlapFeature 0.75 --fracOverlap 0.75 g1.bam g2.bam g3.bam g4.bam p1.bam p2.bam p3.bam &
!sed 's/;gene//' lnc_monoexon75 > lnc_monoexon75.tsv

In [18]:
fc = pd.read_csv('lnc_monoexon75.tsv', sep='\t')
fc.head()

,Geneid,Chr,Start,End,Strand,Length,/home/andrey/anna/g1.bam,/home/andrey/anna/g2.bam,/home/andrey/anna/g3.bam,/home/andrey/anna/g4.bam,/home/andrey/anna/p1.bam,/home/andrey/anna/p2.bam,/home/andrey/anna/p3.bam
0,lnc_tr17504,Chr01,185390,186013,+,624,0,0,0,0,0,0,0
1,lnc_tr4790,Chr01,237391,237672,-,282,0,0,0,0,0,0,0
2,lnc_tr4852,Chr01,627880,628299,+,420,0,0,0,0,0,0,0
3,lnc_tr14183,Chr01,627880,628299,+,420,0,0,0,0,0,0,0
4,lnc_tr14184,Chr01,627995,628180,+,186,0,0,0,0,0,0,0


In [19]:
fc["summa"] = fc.iloc[:,6] + fc.iloc[:,7] + fc.iloc[:,8] + fc.iloc[:,9] + fc.iloc[:,10] + fc.iloc[:,11] + fc.iloc[:,12]
# only identified isoforms
fc = fc[fc.summa > 0]
fc.shape

(1347, 14)

In [34]:
fc["g1_rpm"] = fc.iloc[:,6]/0.393
fc["g2_rpm"] = fc.iloc[:,7]/0.982
fc["g3_rpm"] = fc.iloc[:,8]/0.77
fc["g4_rpm"] = fc.iloc[:,9]/0.88
fc["p1_rpm"] = fc.iloc[:,10]/1.597
fc["p2_rpm"] = fc.iloc[:,11]/0.5656
fc["p3_rpm"] = fc.iloc[:,12]/1.811
fc["source"] = "featureCounts"

In [22]:
fc = fc.rename(columns = {"Geneid": "ids"})
fc = fc.merge(names, how="inner", on="ids")
fc.head()

,ids,Chr,Start,End,Strand,Length,/home/andrey/anna/g1.bam,/home/andrey/anna/g2.bam,/home/andrey/anna/g3.bam,/home/andrey/anna/g4.bam,...,summa,g1_rpm,g2_rpm,g3_rpm,g4_rpm,p1_rpm,p2_rpm,p3_rpm,names_x,names_y
0,lnc_tr9777,Chr01,1852341,1854241,-,1901,1,1,9,7,...,19,2.544529,1.018330,11.688312,7.954545,0.626174,0.0,0.0,>PACX32971569 gene_name=Pp3c1_2520 ref_gene_id...,>PACX32971569 gene_name=Pp3c1_2520 ref_gene_id...
1,lnc_tr4686,Chr01,2764720,2765825,-,1106,0,0,0,1,...,1,0.000000,0.000000,0.000000,1.136364,0.000000,0.0,0.0,>lcl|Ppatens_Pp3c1_3540V3.1,>lcl|Ppatens_Pp3c1_3540V3.1
2,lnc_tr14192,Chr01,2764720,2765825,-,1106,0,0,0,1,...,1,0.000000,0.000000,0.000000,1.136364,0.000000,0.0,0.0,>PACX32968253 gene_name=Pp3c1_3540 ref_gene_id...,>PACX32968253 gene_name=Pp3c1_3540 ref_gene_id...
3,lnc_tr4772,Chr01,3100347,3101036,+,690,3,10,10,10,...,33,7.633588,10.183299,12.987013,11.363636,0.000000,0.0,0.0,>lcl|Ppatens_Pp3c1_4120V3.1,>lcl|Ppatens_Pp3c1_4120V3.1
4,lnc_tr763,Chr01,3292977,3294719,-,1743,0,0,0,1,...,1,0.000000,0.000000,0.000000,1.136364,0.000000,0.0,0.0,>CNT2064367,>CNT2064367


In [25]:
# Now we can concatenate the two tables
flair_cut = flair.drop(columns = ["g1_gam", "g2_gam", "g3_gam","g4_gam", "p1_prot", "p2_prot","p3_prot", "ids"])
fc_cut = fc.iloc[:, 14:]
outp = pd.concat([flair_cut, fc_cut])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


## Antisence

Reads from the opposite strand were taken in a similar way with featureCounts.

#### ./featureCounts -a lnc_annot.saf -F SAF -s 2 -L -O -o lnc_anti_newdata g1.bam g2.bam g3.bam g4.bam p1.bam p2.bam p3.bam &

In [26]:
anti = pd.read_csv("lnc_anti_newdata.tsv", sep="\t", header=None)
anti = anti.rename(columns={0: "ids"})
anti.head()

,ids,1,2,3,4,5,6,7,8,9,10,11,12
0,lnc_tr17504,Chr01,185390,186013,+,624,0,0,0,0,0,0,1
1,lnc_tr4769,Chr01,201028,201445,+,418,0,0,0,0,0,0,0
2,lnc_tr4790,Chr01,237391,237672,-,282,0,0,0,0,0,0,0
3,lnc_tr4852,Chr01,627880,628299,+,420,0,0,0,0,0,0,0
4,lnc_tr14183,Chr01,627880,628299,+,420,0,0,0,0,0,0,0


In [27]:
anti =  anti.merge(names, how="inner", on="ids")

In [28]:
anti["g1_anti_rpm"] = anti.iloc[:,6]/0.393
anti["g2_anti_rpm"] = anti.iloc[:,7]/0.982
anti["g3_anti_rpm"] = anti.iloc[:,8]/0.77
anti["g4_anti_rpm"] = anti.iloc[:,9]/0.88
anti["p1_anti_rpm"] = anti.iloc[:,10]/1.597
anti["p2_anti_rpm"] = anti.iloc[:,11]/0.5656
anti["p3_anti_rpm"] = anti.iloc[:,12]/1.811
anti.head()

,ids,1,2,3,4,5,6,7,8,9,...,11,12,names,g1_anti_rpm,g2_anti_rpm,g3_anti_rpm,g4_anti_rpm,p1_anti_rpm,p2_anti_rpm,p3_anti_rpm
0,lnc_tr17504,Chr01,185390,186013,+,624,0,0,0,0,...,0,1,>Pp3c1_350V3::Chr01:185389-186013(+),0.0,0.0,0.0,0.0,0.0,0.0,0.552181
1,lnc_tr4769,Chr01,201028,201445,+,418,0,0,0,0,...,0,0,>lcl|Ppatens_Pp3c1_410V3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,lnc_tr4790,Chr01,237391,237672,-,282,0,0,0,0,...,0,0,>lcl|Ppatens_Pp3c1_500V3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,lnc_tr4852,Chr01,627880,628299,+,420,0,0,0,0,...,0,0,>lcl|Ppatens_Pp3c1_870V3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,lnc_tr14183,Chr01,627880,628299,+,420,0,0,0,0,...,0,0,>PACX32968721 gene_name=Pp3c1_870 ref_gene_id=...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [29]:
anti = anti.drop(columns=[1,2,3,4,5,6,7,8,9,10,11,12])
anti.head()

,ids,names,g1_anti_rpm,g2_anti_rpm,g3_anti_rpm,g4_anti_rpm,p1_anti_rpm,p2_anti_rpm,p3_anti_rpm
0,lnc_tr17504,>Pp3c1_350V3::Chr01:185389-186013(+),0.0,0.0,0.0,0.0,0.0,0.0,0.552181
1,lnc_tr4769,>lcl|Ppatens_Pp3c1_410V3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,lnc_tr4790,>lcl|Ppatens_Pp3c1_500V3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,lnc_tr4852,>lcl|Ppatens_Pp3c1_870V3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,lnc_tr14183,>PACX32968721 gene_name=Pp3c1_870 ref_gene_id=...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [36]:
# Now we merge all the tables

features = outp.merge(anti, how="outer", on="names")
features = features.drop(columns = ["names_x", "names_y", "ids"])
features.head()

,g1_rpm,g2_rpm,g3_rpm,g4_rpm,names,p1_rpm,p2_rpm,p3_rpm,g1_anti_rpm,g2_anti_rpm,g3_anti_rpm,g4_anti_rpm,p1_anti_rpm,p2_anti_rpm,p3_anti_rpm
0,0.0,0.00000,1.298701,0.0,>XR_002972246.1 rna-XR_002972246.1_NCBI_ncRNA,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
1,0.0,3.05499,0.000000,0.0,>XR_002972246.1 rna-XR_002972246.1_NCBI_ncRNA,0.626174,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,0.0,0.00000,0.000000,0.0,>lcl|Ppatens_Pp3c1_40640V3.1,0.626174,0.0,0.552181,0.0,0.0,0.0,0.0,0.626174,0.0,0.552181
3,0.0,0.00000,9.090909,0.0,>XR_002970609.1 rna-XR_002970609.1_NCBI_ncRNA,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
4,0.0,0.00000,0.000000,0.0,>lcl|Ppatens_Pp3c6_15880V3.1,0.626174,0.0,0.552181,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000


In [37]:
# Adding locuses of each transcript
locuses = pd.read_csv('names_table_locus.csv', sep="\t")
locuses.head()

,transcript,locus
0,XR_002967627.1,"ref|XR_002967596.1|,ref|XR_002967627.1|::Chr22..."
1,lcl|Ppatens_Pp3c2_1700V3.1,"ref|XR_002973712.1|,lcl|Ppatens_Pp3c2_1700V3.1..."
2,lcl|Ppatens_Pp3c20_3270V3.2,"lcl|Ppatens_Pp3c20_3270V3.18474,lcl|Ppatens_Pp..."
3,XR_002967369.1,"ref|XR_002967365.1|,ref|XR_002967366.1|,ref|XR..."
4,lcl|Ppatens_Pp3c13_20260V3.1,lcl|Ppatens_Pp3c13_20260V3.15922::Chr13:142488...


In [41]:
features.to_csv("features.tsv", index=False)
!sed 's/>//' features.tsv > features_corrected.tsv
features = pd.read_csv("features_corrected.tsv")
features = features.rename(columns={"names": "transcript"})
features = features.merge(locuses, how="inner", on="transcript")
features.head()

,g1_rpm,g2_rpm,g3_rpm,g4_rpm,transcript,p1_rpm,p2_rpm,p3_rpm,source,locus,g1_rpm_anti,g2_rpm_anti,g3_rpm_anti,g4_rpm_anti,p1_rpm_anti,p2_rpm_anti,p3_rpm_anti
0,0.000000,0.0,0.000000,0.000000,lcl|Ppatens_Pp3c1_40640V3.1,0.626174,0.000000,0.552181,flair,"lcl|Ppatens_Pp3c1_40640V3.1580,PACX32971456::C...",0.0,0.0,0.0,0.0,0.626174,0.0,0.552181
1,0.000000,0.0,0.000000,0.000000,lcl|Ppatens_Pp3c6_15880V3.1,0.626174,0.000000,0.552181,flair,"ref|XR_002970176.1|,ref|XR_002970178.1|,ref|XR...",0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,2.544529,0.0,2.597403,0.000000,CNT2063702,2.504696,3.536068,0.552181,flair,"lcl|Ppatens_Pp3c7_17330V3.23404,lcl|Ppatens_Pp...",0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
3,0.000000,0.0,0.000000,0.000000,CNT2063702,0.000000,3.536068,0.000000,featureCounts,"lcl|Ppatens_Pp3c7_17330V3.23404,lcl|Ppatens_Pp...",0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
4,0.000000,0.0,3.896104,5.681818,lcl|Ppatens_Pp3c6_6770V3.4,6.887915,3.536068,5.521811,flair,"lcl|Ppatens_Pp3c6_6770V3.42820,PACX32975550::C...",0.0,0.0,0.0,0.0,0.000000,0.0,0.000000


In [ ]:
#Then just changing the columns order
!awk '{print $5, $10, $1, $2, $3, $4, $6, $7, $8, $9, $11, $12, $13, $14, $15, $16, $17}' lnc_nanopore_all.tsv > lnc_nanopore_all_inorder.tsv

## Fasta output

Found isoforms were converted into fasta. Flair gives fasta as an output, we had only to rename the sequences.

In [ ]:
pat = re.compile(r'lnc_tr\d*')
names = pd.read_csv('lnc_tr_names_tomynames.csv', sep=';', header=None)

input = SeqIO.parse("lnc.isoforms.fa", "fasta")

with open("lnc_names.fasta", "w") as outp:
    for i in input:
        for a in pat.findall(i.name):
            print(names.loc[:, 0][names.loc[:, 1] == a])
            i.id =  list(names.loc[:, 0][names.loc[:, 1] == a])[0]
            i.description = "mossss"
            print(i)
            SeqIO.write(i, outp, "fasta")
            
!sed 's/mossss//' lnc_names.fasta > lnc_multiexon.fasta

In [ ]:
# For monoexon features intersect was used.
!bedtools merge -s -i guppy4all_sort.bed > guppy4all_sort_merged.bed 
!bedtools intersect  -s -loj  -a guppy4all_sort_merged.bed -b  lnc_annot_1ex.gff | grep gene_id > lnc_true_s.tsv 
!awk '{print $1, "\t", $2,"\t", $3,"\t", $6,"\t", $4}' lnc_true_s.bed | sed 's/ //g' > lnc_true_s_ord.bed
!bedtools getfasta -name -fi nanomoss.fasta -bed lnc_true_s_ord.bed -fo lnc_monoex1.fasta
#Then renaming as given above.

## Stringtie

Stringtie (https://github.com/gpertea/stringtie) was also used to prove lncRNA isoforms expression. We used both Flair and Stringtie results to compare with annotation.
Gffcompare (https://github.com/gpertea/gffcompare) codes of .gtf.refmap output file indicate type of intersection with the annotation.

In [ ]:
!./stringtie -L -G lnc_annot.gtf -B -o stringtie_lnc.gtf lnc.bam # all reads aligned in one .bam file, produced by Flair

!./gffcompare -R -r lnc_annot.gtf -o compare_lnc_stringtie stringtie_lnc.gtf
!./gffcompare -R -r lnc_annot.gtf -o compare_lnc_flair lnc.gtf